In [83]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from numba import jit

In [94]:
x_dim = 5 # dimension of observations
num_MBONS = 1
freqs = [0.5, 0.5]
mus = []

for i in range(num_MBONS+1):
    mus.append(5*np.random.randn(x_dim,1))
    
sig = np.random.randn(x_dim,x_dim)/np.sqrt(x_dim)
Sigma = sig@sig.T
samples = 100000 # number of data points

In [95]:
classes = []
for i in range(num_MBONS+1):
    classes.append(mus[i] + sig@np.random.randn(x_dim,int(samples*freqs[i])))

In [96]:
X = np.concatenate(classes, axis=1)

In [97]:

"""
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA

# import some data to play with
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
            edgecolor='k')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()
"""

"\nfrom mpl_toolkits.mplot3d import Axes3D\nfrom sklearn import datasets\nfrom sklearn.decomposition import PCA\n\n# import some data to play with\niris = datasets.load_iris()\nX = iris.data  # we only take the first two features.\ny = iris.target\n\nx_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5\ny_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5\n\nplt.figure(2, figsize=(8, 6))\nplt.clf()\n\n# Plot the training points\nplt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,\n            edgecolor='k')\nplt.xlabel('Sepal length')\nplt.ylabel('Sepal width')\n\nplt.xlim(x_min, x_max)\nplt.ylim(y_min, y_max)\nplt.xticks(())\nplt.yticks(())\nplt.show()\n"

In [98]:
"""
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=110)
X_reduced = PCA(n_components=3).fit_transform(iris.data)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y,
           cmap=plt.cm.Set1, edgecolor='k', s=40)
ax.set_title("First three PCA directions")
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])

plt.show()
"""

'\nfig = plt.figure(1, figsize=(8, 6))\nax = Axes3D(fig, elev=-150, azim=110)\nX_reduced = PCA(n_components=3).fit_transform(iris.data)\nax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y,\n           cmap=plt.cm.Set1, edgecolor=\'k\', s=40)\nax.set_title("First three PCA directions")\nax.set_xlabel("1st eigenvector")\nax.w_xaxis.set_ticklabels([])\nax.set_ylabel("2nd eigenvector")\nax.w_yaxis.set_ticklabels([])\nax.set_zlabel("3rd eigenvector")\nax.w_zaxis.set_ticklabels([])\n\nplt.show()\n'

In [128]:
x_dim = 5
k = 2
T = 150
theta = np.zeros((k, 1))
n = np.ones((k,1))
Y=np.zeros((k,T))

In [129]:

for epochs in range(20):
    w = (np.random.randn(x_dim,k))

    idx = np.random.permutation(T)
    X_ = X[:,idx]
    labels = iris.target
    labels = labels[idx]

    for t in range(T):
        x = X_[None, :, t].T
        
        a = -w.T@x + theta

        c = np.argmin(a)
        z = -a[c]
        y = np.zeros((k, 1))
        y[c] = 1
        
        n += y
        for i in range(k):
            w[:,i] += 2*y[i]*(x.reshape(-1)-w[:,i])/n[i]
            theta[i] += 2*y[i]*(z-theta[i])/n[i]
    
    
        #w += (2*x.dot(np.ones((1,k))) - w)*np.diag(y/n)
        #theta += np.diag(y/n) * (z - theta)
        Y[:,t] = y.reshape(-1)

In [130]:
w

array([[-0.85392853, -1.10945019],
       [-0.4573836 ,  1.80213421],
       [-0.59454223,  0.47689125],
       [-0.01121998, -1.42786369],
       [-0.24916799,  1.43198317]])

In [131]:
Y, np.argmax(Y, axis=0)

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 

In [132]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_.T)
kmeans.cluster_centers_.T

array([[-0.95535308, -0.26136482],
       [ 0.76244971,  1.4974539 ],
       [ 5.1051348 ,  3.84708562],
       [-5.27466281, -6.31511704],
       [ 6.55740322,  4.95958957]])

In [133]:
kmeans.labels_

array([0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

In [14]:
    #er = 1;
    #i_iter = 1;
    
    #while er > 1e-7:
            
         #y = np.maximum(0, y + (eta)*(w.T@x + z + rho*(np.sum(y) - 1) ) )
         #z = z + eta*( np.sum(y) - 1 )  

         #er = max(np.norm(z_prev-z)/(np.norm(z_prev)+1e-4),np.norm(y_prev-y)/(np.norm(y_prev)+1e-4));
         #iter=iter+1